# MultiCompartment

## 电生理特性

在真实神经元中，细胞不仅仅是一个点状结构，而是具有复杂的空间结构：树突、轴突、胞体等，这些结构对神经元的电生理行为有显著影响。
为了更精确地模拟空间上的电信号传播，我们引入了 `MultiCompartment` 。

在此之前，我们有必要向你介绍电缆理论，以方便你理解后续内容。

实际上，神经元并非只是简单的电容器，而是具有空间结构的电缆型系统。电缆理论是一种用于描述电信号在神经元轴突或树突等细长结构中如何传播的理论模型。
电缆理论源于工程学中对传输线的建模，在神经科学中，它将树突和轴突看作一维电缆，能够模拟电信号随时间和空间的变化。

计算模型中，电缆理论通常通过离散房室模型来实现。神经元被分割成多个房室，每个房室是一个等效电路单元，相邻房室之间通过电阻连接，形成近似的空间结构模拟。
结合我们的建模框架不难发现，这就是 `MultiCompartment` 建模的核心原理。

在这种建模方式中，整个神经元被划分为多个空间上连接的房室，每个房室有独立的膜电位、膜电流，并通过电缆理论中的轴向电阻进行耦合。
- 每个房室拥有自己的离子通道分布和膜电流。
- 房室之间通过电阻连接，电流可以在房室之间传导。
- 能够描述电信号在树突或轴突上的衰减、延迟、反射等现象。

其电学形式遵循电缆方程：

$$
C_m \frac{dV_i}{dt} = -I_{\text{ion}, i} + \sum_j \frac{(V_j - V_i)}{R_{ij}} + I_{\text{ext}, i}
$$

其中：

- $C_m$：膜电容
- $I_{\text{ion}, i}$：第 $i$ 个房室的离子电流
- $\frac{(V_j - V_i)}{R_{ij}}$：从相邻房室 $j$ 传导到房室 $i$ 的轴向电流
- $R_{ij}$：两个房室之间的轴向电阻
- $I_{\text{ext}, i}$：外部注入电流

由于具备空间建模能力，多房室模型广泛用于建模复杂树突整合、动作电位沿轴突传播等电生理现象。

## 建模实现

同样，`MultiCompartment` 的建模也并不复杂，整体思路与 `SingleCompartment` 的建模类似，只是我们需要定义的参数不同了。

我们来看这个例子：

```python
class MultiCompartmentHH(braincell.MultiCompartment):
    def __init__(self,  size, connection, Ra, cm, diam, L, solver = 'exp_euler'):
        super().__init__(
            size=size,
            connection= connection,
            Ra= Ra * u.ohm * u.cm,
            cm= cm * u.uF / u.cm ** 2,
            diam=diam * u.um,
            L= L * u.um,
            V_th=20. * u.mV,
            V_initializer=brainstate.init.Constant(-65 * u.mV),
            spk_fun=brainstate.surrogate.ReluGrad(),
            solver = solver
        )
        self.IL = braincell.channel.IL(self.varshape, E= -65 * u.mV, g_max= 0.001 * s)
```

这个例子展示了如何使用 `MultiCompartment` 构建一个简单的多房室神经元模型，设定相关参数，并且添加了一个漏电流通道。当然，这只是一个很简单的例子，我们可以对模型进一步拓展，添加其他通道或者注入电流。

通过 `MultiCompartment`，我们可以轻松构建结构精细、行为多样的多房室神经元模型。